In [1]:
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import numpy as np
import os

In [2]:
def extract(path, include_df=False):
    df = pd.concat([pd.read_csv(os.path.join(path, file)) for file in os.listdir(path)])
    features = df.copy()
    labels = features.pop('agent_action')
    return (np.array(features), np.array(labels)) if not include_df else (df, np.array(features), np.array(labels))

In [3]:
path = 'ftg-data/Dataset/BlackMamba/'

df, features, labels = extract(os.path.join(path, 'export'), include_df=True)

In [4]:
class_names = ['neutral', 'stand', 'forward_walk', 'dash', 'back_step', 'crouch', 'jump', 'for_jump', 'back_jump', 'air'
             , 'stand_guard', 'crouch_guard', 'air_guard', 'stand_guard_recov', 'crouch_guard_recov', 'air_guard_recov'
             , 'stand_recov', 'crouch_recov', 'air_recov', 'change_down', 'down', 'rise', 'landing', 'throw_a', 'throw_b'
             , 'throw_hit', 'throw_suffer', 'stand_a', 'stand_b', 'crouch_a', 'crouch_b', 'air_a', 'air_b', 'air_da', 'air_db'
             , 'stand_fa', 'stand_fb', 'crouch_fa', 'crouch_fb', 'air_fa', 'air_fb', 'air_ua', 'air_ub', 'stand_d_df_fa'
             , 'stand_d_df_fb', 'stand_f_d_dfa', 'stand_f_d_dfb', 'stand_d_db_ba', 'stand_d_db_bb', 'air_d_df_fa'
             , 'air_d_df_fb', 'air_f_d_dfa', 'air_f_d_dfb', 'air_d_db_ba', 'air_d_db_bb', 'stand_d_df_fc']

model = tf.keras.models.load_model(os.path.join(path, 'saved_model/model.h5'))

In [5]:
predicted = [np.argmax(x) for x in model.predict(features)]
# df = df.drop(columns=['target_selected_action'])
export_df = pd.DataFrame()
export_df['actual_action'] = [class_names[x] for x in labels]
export_df['predicted_action'] = [class_names[x] for x in predicted]
export_df.to_csv(os.path.join(path, 'compare.csv'))
export_df

,actual_action,predicted_action
0,stand,stand
1,crouch,crouch
2,crouch,crouch
3,crouch,crouch
4,crouch,crouch
...,...,...
1770,crouch_b,crouch_b
1771,crouch_b,crouch_b
1772,crouch_b,crouch_b
1773,crouch_b,crouch_b


In [6]:
action_labels = list(set(labels))
class_names = [class_names[x] for x in action_labels]
report = classification_report(labels, predicted, labels=action_labels, target_names=class_names, zero_division=0)
print(report)

               precision    recall  f1-score   support

        stand       0.95      1.00      0.97       134
 forward_walk       1.00      0.81      0.89        26
         dash       0.80      1.00      0.89        12
       crouch       0.71      0.98      0.82        42
     for_jump       1.00      1.00      1.00        13
          air       1.00      0.96      0.98        48
 crouch_guard       0.85      0.97      0.90        29
  stand_recov       1.00      1.00      1.00        75
 crouch_recov       1.00      1.00      1.00        43
  change_down       1.00      1.00      1.00        30
         down       1.00      1.00      1.00       126
         rise       1.00      1.00      1.00       174
      landing       1.00      1.00      1.00        24
 throw_suffer       0.94      1.00      0.97        58
      stand_a       1.00      0.88      0.94        17
      stand_b       1.00      1.00      1.00        82
     crouch_a       1.00      0.94      0.97        36
     crou